In [1]:
import xgboost
import sklearn

print("xgboost version:", xgboost.__version__)
print("scikit-learn version:", sklearn.__version__)


xgboost version: 2.1.1
scikit-learn version: 1.6.1


In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime as dt
import itertools

%matplotlib inline

Reading data from the CSV into a dataframe

In [ ]:
folder='./Datasets/'
raw_data_1 = pd.read_csv(folder+'2000-01.csv')
raw_data_2 = pd.read_csv(folder+'2001-02.csv')
raw_data_3 = pd.read_csv(folder+'2002-03.csv') 
raw_data_4 = pd.read_csv(folder+'2003-04.csv')
raw_data_5 = pd.read_csv(folder+'2004-05.csv')
raw_data_6 = pd.read_csv(folder+'2005-06.csv')
raw_data_7 = pd.read_csv(folder+'2006-07.csv')
raw_data_8 = pd.read_csv(folder+'2007-08.csv')
raw_data_9 = pd.read_csv(folder+'2008-09.csv')
raw_data_10 = pd.read_csv(folder+'2009-10.csv')
raw_data_11 = pd.read_csv(folder+'2010-11.csv')
raw_data_12 = pd.read_csv(folder+'2011-12.csv')
raw_data_13 = pd.read_csv(folder+'2012-13.csv')
raw_data_14 = pd.read_csv(folder+'2013-14.csv')
raw_data_15 = pd.read_csv(folder+'2014-15.csv')
raw_data_16 = pd.read_csv(folder+'2015-16.csv')
raw_data_17 = pd.read_csv(folder+'2016-17.csv')
raw_data_18 = pd.read_csv(folder+'2017-18.csv')
raw_data_19 = pd.read_csv(folder+'2018-19.csv')
raw_data_20 = pd.read_csv(folder+'2019-20.csv')
raw_data_21 = pd.read_csv(folder+'2020-2021.csv')
raw_data_22 = pd.read_csv(folder+'2021-2022.csv')

In [ ]:
print(len(raw_data_19))


In [ ]:
#Gets all the statistics related to the gameplay
columns = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
playing_stats_1 = raw_data_1[columns]
playing_stats_2 = raw_data_2[columns]
playing_stats_3 = raw_data_3[columns]
playing_stats_4 = raw_data_4[columns]
playing_stats_5 = raw_data_5[columns]
playing_stats_6 = raw_data_6[columns]
playing_stats_7 = raw_data_7[columns]
playing_stats_8 = raw_data_8[columns]
playing_stats_9 = raw_data_9[columns]
playing_stats_10 = raw_data_10[columns]
playing_stats_11 = raw_data_11[columns]
playing_stats_12 = raw_data_12[columns]
playing_stats_13 = raw_data_13[columns]
playing_stats_14 = raw_data_14[columns]
playing_stats_15 = raw_data_15[columns]
playing_stats_16 = raw_data_16[columns]
playing_stats_17 = raw_data_17[columns]
playing_stats_18 = raw_data_18[columns]
#playing_stats_19 = raw_data_19[columns]
playing_stats_20 = raw_data_20[columns]
playing_stats_21 = raw_data_21[columns]
playing_stats_22 = raw_data_22[columns]


In [ ]:
playing_stats_1 

In [ ]:
#Gets the goals agg arranged by teams
def get_goals_agg(playing_stats):
    #Created a dictionary with the teams as keys and the goals scored as values
    teams = {}
    for i in playing_stats.groupby('HomeTeam').mean(numeric_only=True).T.columns:
        teams[i] = []
    for i in range(len(playing_stats)):
        teams[playing_stats['HomeTeam'][i]].append(playing_stats['FTHG'][i])
        teams[playing_stats['AwayTeam'][i]].append(playing_stats['FTAG'][i])
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsScored[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
    return GoalsScored

In [ ]:
def get_home_goals_19(playing_stats):
    """
    Returns individual home match goals for each team with their opponents.
    
    Parameters:
    -----------
    playing_stats : DataFrame
        DataFrame containing match statistics with columns:
        - HomeTeam: home team name
        - AwayTeam: away team name
        - FTHG: full time home goals
        - Date: match date (optional, used for sorting)
    
    Returns:
    --------
    DataFrame
        DataFrame containing home matches with columns:
        - HomeTeam: home team name
        - AwayTeam: away team name
        - Goals: goals scored by home team
    """
    # Verify required columns
    required_cols = {'HomeTeam', 'AwayTeam', 'FTHG'}
    if not required_cols.issubset(playing_stats.columns):
        raise ValueError("DataFrame must contain 'HomeTeam', 'AwayTeam', and 'FTHG' columns")
    
    # Create a copy and sort by date if available
    stats = playing_stats.copy()
    if 'Date' in stats.columns:
        stats = stats.sort_values('Date')
    
    # Create lists to store the results
    home_teams = []
    away_teams = []
    goals = []
    
    # Process matches for each team
    for team in stats['HomeTeam'].unique():
        # Get home matches for this team
        team_matches = stats[stats['HomeTeam'] == team].head(19)
        
        # Add match details to lists
        for _, match in team_matches.iterrows():
            home_teams.append(match['HomeTeam'])
            away_teams.append(match['AwayTeam'])
            goals.append(match['FTHG'])
    
    # Create and return the DataFrame
    result_df = pd.DataFrame({
        'HomeTeam': home_teams,
        'AwayTeam': away_teams,
        'Goals': goals
    })
    
    return result_df

In [ ]:
# Test de la fonction avec playing_stats_1
test_df = get_home_goals_19(playing_stats_1)
# Filter matches where Man City is the home team
man_city_home = test_df[test_df['HomeTeam'] == 'Man City']
print("Manchester City's home matches:")
print(man_city_home)

In [ ]:
def get_goals_conceded(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean(numeric_only=True).T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        ATGC = playing_stat.iloc[i]['FTHG']
        HTGC = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGC)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGC)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsConceded[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
    return GoalsConceded

In [ ]:
def get_gss(playing_stat):
    GC = get_goals_conceded(playing_stat)
    GS = get_goals_agg(playing_stat)
   
    j = 0
    HTGS = []
    ATGS = []
    HTGC = []
    ATGC = []

    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])
        
        if ((i + 1)% 10) == 0:
            j = j + 1
    playing_stat = playing_stat.copy()
  
    playing_stat['HTGS'] = HTGS
    playing_stat['ATGS'] = ATGS
    playing_stat['HTGC'] = HTGC
    playing_stat['ATGC'] = ATGC
    
    return playing_stat


In [ ]:
playing_stats_1 = get_gss(playing_stats_1)
playing_stats_2 = get_gss(playing_stats_2)
playing_stats_3 = get_gss(playing_stats_3)
playing_stats_4 = get_gss(playing_stats_4)
playing_stats_5 = get_gss(playing_stats_5)
playing_stats_6 = get_gss(playing_stats_6)
playing_stats_7 = get_gss(playing_stats_7)
playing_stats_8 = get_gss(playing_stats_8)
playing_stats_9 = get_gss(playing_stats_9)
playing_stats_10 = get_gss(playing_stats_10)
playing_stats_11 = get_gss(playing_stats_11)
playing_stats_12 = get_gss(playing_stats_12)
playing_stats_13 = get_gss(playing_stats_13)
playing_stats_14 = get_gss(playing_stats_14)
playing_stats_15 = get_gss(playing_stats_15)
playing_stats_16 = get_gss(playing_stats_16)
playing_stats_17 = get_gss(playing_stats_17)
playing_stats_18 = get_gss(playing_stats_18)
#playing_stats_19 = get_gss(playing_stats_19)
#playing_stats_20 = get_gss(playing_stats_20)
playing_stats_21 = get_gss(playing_stats_21)
playing_stats_22 = get_gss(playing_stats_22)

In [ ]:
playing_stats_1

In [ ]:
def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0

In [ ]:
def get_cuml_points(matchres):
    matchres_points = matchres.apply(lambda col: col.map(get_points))
    
    for i in range(2, 39):
        matchres_points[i] = matchres_points[i] + matchres_points[i - 1]
    
    matchres_points.insert(loc=0, column=0, value=[0] * matchres.shape[0])
    return matchres_points


In [ ]:
def get_matchres(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean(numeric_only=True).T.columns:
        teams[i] = []

    # the value corresponding to keys is a list containing the match result
    for i in range(len(playing_stat)):
        if playing_stat.iloc[i].FTR == 'H':
            teams[playing_stat.iloc[i].HomeTeam].append('W')
            teams[playing_stat.iloc[i].AwayTeam].append('L')
        elif playing_stat.iloc[i].FTR == 'A':
            teams[playing_stat.iloc[i].AwayTeam].append('W')
            teams[playing_stat.iloc[i].HomeTeam].append('L')
        else:
            teams[playing_stat.iloc[i].AwayTeam].append('D')
            teams[playing_stat.iloc[i].HomeTeam].append('D')
            
    return pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T

In [ ]:
matchres = get_matchres(playing_stats_1)
cum_pts = get_cuml_points(matchres)
cum_pts

In [ ]:
get_matchres(playing_stats_1)

In [ ]:
def get_agg_points(playing_stat):
    matchres = get_matchres(playing_stat)
    cum_pts = get_cuml_points(matchres)
    HTP = []
    ATP = []
    j = 0
    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])

        if ((i + 1)% 10) == 0:
            j = j + 1
    playing_stat = playing_stat.copy()
    playing_stat['HTP'] = HTP
    playing_stat['ATP'] = ATP
    return playing_stat

In [ ]:
playing_stats_1 = get_agg_points(playing_stats_1)
playing_stats_2 = get_agg_points(playing_stats_2)
playing_stats_3 = get_agg_points(playing_stats_3)
playing_stats_4 = get_agg_points(playing_stats_4)
playing_stats_5 = get_agg_points(playing_stats_5)
playing_stats_6 = get_agg_points(playing_stats_6)
playing_stats_7 = get_agg_points(playing_stats_7)
playing_stats_8 = get_agg_points(playing_stats_8)
playing_stats_9 = get_agg_points(playing_stats_9)
playing_stats_10 = get_agg_points(playing_stats_10)
playing_stats_11 = get_agg_points(playing_stats_11)
playing_stats_12 = get_agg_points(playing_stats_12)
playing_stats_13 = get_agg_points(playing_stats_13)
playing_stats_14 = get_agg_points(playing_stats_14)
playing_stats_15 = get_agg_points(playing_stats_15)
playing_stats_16 = get_agg_points(playing_stats_16)
playing_stats_17 = get_agg_points(playing_stats_17)
playing_stats_18 = get_agg_points(playing_stats_18)
playing_stats_21 =get_agg_points(playing_stats_21)
playing_stats_22= get_agg_points(playing_stats_22)

In [ ]:
playing_stats_1

The function creates a form summary (like "WWDLD") for each team by aggregating the results over the last num games before each matchday.

In [ ]:
# `playing_stat`: a DataFrame containing football match stats
# `num`: how many previous matchdays to include in the form
def get_form(playing_stat, num):
    # Get match results first
    form = get_matchres(playing_stat)
    form_final = form.copy()
    
    # For each matchday
    for i in range(1, 39):
        # For each team
        for team in form_final.index:
            # Get form for previous num matches
            form_str = ''
            for j in range(min(num, i)):  # Only look at available previous matches
                if i-j-1 >= 0:  # Check if the previous match exists
                    form_str = form.loc[team, i-j] + form_str  # Prepend to keep most recent last
                else:
                    form_str = 'M' + form_str  # Missing data for unavailable matches
            
            # Store the form string
            form_final.loc[team, i] = form_str
            
    return form_final

In [ ]:
get_form(playing_stats_1,3)

This function: Adds two new columns to playing_stat:

HMn: home team’s form on matchday n (most recent result from last num games)

AMn: away team’s form on matchday n (same)

In [ ]:
def add_form(playing_stat, num):
    # Create a copy of the DataFrame to avoid modifying the original
    playing_stat = playing_stat.copy()
    
    # Remove any existing form columns for this number
    form_cols = [f'HM{num}', f'AM{num}']
    playing_stat = playing_stat.drop(columns=form_cols, errors='ignore')
    
    # Get the form data
    form = get_form(playing_stat, num)
    h = []  # Home team form
    a = []  # Away team form
    
    # Process each match
    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        week = (i // 10) + 1  # Calculate the match week (1-38)
        
        # Get form strings for both teams
        if week <= num:
            # Not enough previous matches for full history
            h.append('M')
            a.append('M')
        else:
            # Get the last character (most recent result) from form string
            ht_form = form.loc[ht][week]
            at_form = form.loc[at][week]
            h.append(ht_form[-1] if len(ht_form) > 0 else 'M')
            a.append(at_form[-1] if len(at_form) > 0 else 'M')
    
    # Add the form columns
    playing_stat[f'HM{num}'] = h
    playing_stat[f'AM{num}'] = a
    
    return playing_stat

In [ ]:
# Test add_form uniquement avec num=3
test_df = add_form(playing_stats_1, 2)
test_df 



In [ ]:

# Function to add form history features
def add_form_df(playing_statistics):
    """
    Add form history columns to the dataframe showing last 5 matches for each team
    HM1-HM5: Home team's form for last 5 matches (most recent to oldest)
    AM1-AM5: Away team's form for last 5 matches (most recent to oldest)
    """
    # Make a copy to avoid modifying original
    df = playing_statistics.copy()
    
    # Create team results dictionary to track form
    team_forms = {}
    for team in df['HomeTeam'].unique():
        team_forms[team] = ['M'] * 5  # Initialize with 'M' for first 5 matches
    
    # Remove any existing form columns
    form_cols = [col for col in df.columns if col.startswith('HM') or col.startswith('AM')]
    df = df.drop(columns=form_cols, errors='ignore')
    
    # Initialize form columns
    for i in range(1, 6):
        df[f'HM{i}'] = 'M'
        df[f'AM{i}'] = 'M'
    
    # Process each match sequentially to build form history
    for idx in range(len(df)):
        home_team = df.iloc[idx]['HomeTeam']
        away_team = df.iloc[idx]['AwayTeam']
        result = df.iloc[idx]['FTR']
        
        # Get current form lists for both teams
        home_form = team_forms[home_team]
        away_form = team_forms[away_team]
        
        # Update current row with existing form
        for i in range(5):
            df.at[idx, f'HM{i+1}'] = home_form[i]
            df.at[idx, f'AM{i+1}'] = away_form[i]
        
        # Update form history based on this match result
        if result == 'H':
            # Home win
            home_form = ['W'] + home_form[:-1]
            away_form = ['L'] + away_form[:-1]
        elif result == 'A':
            # Away win
            home_form = ['L'] + home_form[:-1]
            away_form = ['W'] + away_form[:-1]
        else:
            # Draw
            home_form = ['D'] + home_form[:-1]
            away_form = ['D'] + away_form[:-1]
            
        # Store updated form history
        team_forms[home_team] = home_form
        team_forms[away_team] = away_form
        
    return df

In [ ]:
playing_stats_1 = add_form_df(playing_stats_1)
playing_stats_1

In [ ]:
playing_stats_1 = add_form_df(playing_stats_1)
playing_stats_2 = add_form_df(playing_stats_2)
playing_stats_3 = add_form_df(playing_stats_3)
playing_stats_4 = add_form_df(playing_stats_4)
playing_stats_5 = add_form_df(playing_stats_5)
playing_stats_6 = add_form_df(playing_stats_6)
playing_stats_7 = add_form_df(playing_stats_7)
playing_stats_8 = add_form_df(playing_stats_8)
playing_stats_9 = add_form_df(playing_stats_9)
playing_stats_10 = add_form_df(playing_stats_10)
playing_stats_11 = add_form_df(playing_stats_11)
playing_stats_12 = add_form_df(playing_stats_12)
playing_stats_13 = add_form_df(playing_stats_13)
playing_stats_14 = add_form_df(playing_stats_14)
playing_stats_15 = add_form_df(playing_stats_15)
playing_stats_16 = add_form_df(playing_stats_16)
playing_stats_17 = add_form_df(playing_stats_17)
playing_stats_18 = add_form_df(playing_stats_18)
playing_stats_21 = add_form_df(playing_stats_21)
playing_stats_22 = add_form_df(playing_stats_22)


In [ ]:
cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTGS', 'ATGS', 'HTGC', 'ATGC', 'HTP', 'ATP', 'HM1', 'HM2', 'HM3',
        'HM4', 'HM5', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5' ]
playing_stats_1 = playing_stats_1[cols]
playing_stats_2 = playing_stats_2[cols]
playing_stats_3 = playing_stats_3[cols]
playing_stats_4 = playing_stats_4[cols]
playing_stats_5 = playing_stats_5[cols]
playing_stats_6 = playing_stats_6[cols]
playing_stats_7 = playing_stats_7[cols]
playing_stats_8 = playing_stats_8[cols]
playing_stats_9 = playing_stats_9[cols]
playing_stats_10 = playing_stats_10[cols]
playing_stats_11 = playing_stats_11[cols]
playing_stats_12 = playing_stats_12[cols]
playing_stats_13 = playing_stats_13[cols]
playing_stats_14 = playing_stats_14[cols]
playing_stats_15 = playing_stats_15[cols]
playing_stats_16 = playing_stats_16[cols]
playing_stats_17 = playing_stats_17[cols]
playing_stats_18 = playing_stats_18[cols]
playing_stats_21 = playing_stats_21[cols]
playing_stats_22 = playing_stats_22[cols]

In [ ]:
def get_mw(playing_stat):
    j = 1
    MatchWeek = []
    for i in range(380):
        MatchWeek.append(j)
        if ((i + 1)% 10) == 0:
            j = j + 1
    playing_stat['MW'] = MatchWeek
    return playing_stat

In [ ]:
playing_stats_1 = get_mw(playing_stats_1)
playing_stats_2 = get_mw(playing_stats_2)
playing_stats_3 = get_mw(playing_stats_3)
playing_stats_4 = get_mw(playing_stats_4)
playing_stats_5 = get_mw(playing_stats_5)
playing_stats_6 = get_mw(playing_stats_6)
playing_stats_7 = get_mw(playing_stats_7)
playing_stats_8 = get_mw(playing_stats_8)
playing_stats_9 = get_mw(playing_stats_9)
playing_stats_10 = get_mw(playing_stats_10)
playing_stats_11 = get_mw(playing_stats_11)
playing_stats_12 = get_mw(playing_stats_12)
playing_stats_13 = get_mw(playing_stats_13)
playing_stats_14 = get_mw(playing_stats_14)
playing_stats_15 = get_mw(playing_stats_15)
playing_stats_16 = get_mw(playing_stats_16)
playing_stats_17 = get_mw(playing_stats_17)
playing_stats_18 = get_mw(playing_stats_18)
playing_stats_21 = get_mw(playing_stats_21)
playing_stats_22 = get_mw(playing_stats_22)

final dataframe

In [ ]:
playing_stat = pd.concat([playing_stats_1, playing_stats_2, playing_stats_3, playing_stats_4, playing_stats_5, playing_stats_6,
                        playing_stats_7, playing_stats_8, playing_stats_9, playing_stats_10, playing_stats_11, playing_stats_12,
                        playing_stats_13, playing_stats_14, playing_stats_15, playing_stats_16, playing_stats_17, playing_stats_18
                        , playing_stats_21, playing_stats_22], ignore_index=True)

In [ ]:
# Check the last few rows of the DataFrame to see the form patterns
print("Last 5 matches form for home teams:")
print(playing_stats_1[['HomeTeam', 'HM1', 'HM2', 'HM3', 'HM4', 'HM5']].tail(10))

print("\nLast 5 matches form for away teams:")
print(playing_stats_1[['AwayTeam', 'AM1', 'AM2', 'AM3', 'AM4', 'AM5']].tail(10))

# Check if form values are being properly updated
last_match = playing_stat.iloc[-1]
print("\nDetailed form info for last match:")
print(f"Home Team: {last_match['HomeTeam']}")
print(f"Home Form: {last_match['HM1']},{last_match['HM2']},{last_match['HM3']},{last_match['HM4']},{last_match['HM5']}")
print(f"Away Team: {last_match['AwayTeam']}")
print(f"Away Form: {last_match['AM1']},{last_match['AM2']},{last_match['AM3']},{last_match['AM4']},{last_match['AM5']}")

In [ ]:
def get_form_points(string):
    sum = 0
    for letter in string:
        sum += get_points(letter)
    return sum

In [ ]:
playing_stat['HTFormPtsStr'] = playing_stat['HM1'] + playing_stat['HM2'] + playing_stat['HM3'] + playing_stat['HM4'] + playing_stat['HM5']
playing_stat['ATFormPtsStr'] = playing_stat['AM1'] + playing_stat['AM2'] + playing_stat['AM3'] + playing_stat['AM4'] + playing_stat['AM5']

playing_stat['HTFormPts'] = playing_stat['HTFormPtsStr'].apply(get_form_points)
playing_stat['ATFormPts'] = playing_stat['ATFormPtsStr'].apply(get_form_points)
playing_stat 

In [ ]:
def get_3game_ws(string):
    if string[-3:] == 'WWW':
        return 1
    else:
        return 0
    
def get_5game_ws(string):
    if string == 'WWWWW':
        return 1
    else:
        return 0
    
def get_3game_ls(string):
    if string[-3:] == 'LLL':
        return 1
    else:
        return 0
    
def get_5game_ls(string):
    if string == 'LLLLL':
        return 1
    else:
        return 0

In [ ]:
playing_stat['HTWinStreak3'] = playing_stat['HTFormPtsStr'].apply(get_3game_ws)
playing_stat['HTWinStreak5'] = playing_stat['HTFormPtsStr'].apply(get_5game_ws)
playing_stat['HTLossStreak3'] = playing_stat['HTFormPtsStr'].apply(get_3game_ls)
playing_stat['HTLossStreak5'] = playing_stat['HTFormPtsStr'].apply(get_5game_ls)

playing_stat['ATWinStreak3'] = playing_stat['ATFormPtsStr'].apply(get_3game_ws)
playing_stat['ATWinStreak5'] = playing_stat['ATFormPtsStr'].apply(get_5game_ws)
playing_stat['ATLossStreak3'] = playing_stat['ATFormPtsStr'].apply(get_3game_ls)
playing_stat['ATLossStreak5'] = playing_stat['ATFormPtsStr'].apply(get_5game_ls)

In [ ]:
playing_stat.keys()

In [ ]:
# Get Goal Difference
playing_stat['HTGD'] = playing_stat['HTGS'] - playing_stat['HTGC']
playing_stat['ATGD'] = playing_stat['ATGS'] - playing_stat['ATGC']
playing_stat['DiffGD'] = playing_stat['HTGS'] - playing_stat['ATGS']

# Diff in points
playing_stat['DiffPts'] = playing_stat['HTP'] - playing_stat['ATP']
playing_stat['DiffFormPts'] = playing_stat['HTFormPts'] - playing_stat['ATFormPts']

In [ ]:
playing_stat

In [ ]:
playing_stat.to_csv('Final_Dataset.csv', index=False)

In [ ]:
# Test de la fonction avec les données de la première saison
test_df = get_home_goals_19(playing_stats_1)

# Afficher les résultats pour quelques équipes
print("Buts marqués à domicile par match :")
print(test_df.head())

# Vérifier les détails des buts pour une équipe spécifique
team = test_df['HomeTeam'].unique()[0]
print(f"\nDétails des buts pour {team}:")
print(test_df[test_df['HomeTeam'] == team])

Load Data

In [ ]:
df = pd.read_csv('Final_Dataset.csv')
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.isna().sum()

Encode categorical data

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import (
    f_classif, chi2, mutual_info_classif, SelectKBest
)
from sklearn.pipeline import Pipeline
# Create an instance of OneHotEncoder
encoder = OneHotEncoder(dtype=int, sparse_output=True)

In [ ]:
# Fit and transform the categorical columns
encoded_array = encoder.fit_transform(df[["FTR"]])
# Get correct feature names using get_feature_names_out()
encoded_columns = encoder.get_feature_names_out(["FTR"])
#Convert the encoded data into a DataFrame with the correct column names
nominal = pd.DataFrame(encoded_array.toarray(),
                       columns=encoded_columns) 
print(encoded_array.toarray())
print(nominal)

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(df.select_dtypes(include="number").corr(),annot=True)
plt.show()

In [ ]:
#Features engineering
y_label = df['FTR']
X = df.copy().drop(columns=['FTR'])

In [ ]:
num_cols = X.select_dtypes(include='number').columns.to_list()
cat_cols = X.select_dtypes(exclude='number').columns.to_list()
print("Numériques :", num_cols)
print("Catégorielles :", cat_cols)

In [ ]:
le = LabelEncoder()
y_encoded = le.fit_transform(y_label)  
y_encoded  


In [ ]:
F, pvals = f_classif(X[num_cols], y_encoded)
anova_res = pd.Series(pvals, index=num_cols).sort_values()
print("p-values (ANOVA) les plus petites :")
print(anova_res.head(10))


In [ ]:
selectdata = df.copy().drop(columns=['Date','HomeTeam','AwayTeam','FTHG','FTAG','HTGS','ATGS','HTGC','ATGC','HM4','HM5','AM4','AM5','MW','HTFormPtsStr','ATFormPtsStr','HTWinStreak3',
       'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5', 'ATWinStreak3','ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5'])

In [ ]:
selectdata.keys()

In [ ]:
selectdata.tail()

In [ ]:
n_features = selectdata.shape[1] -1
n_matches = selectdata.shape[0]
print("Total number of matches: {}".format(n_matches))
print ("Number of features: {}".format(n_features))


In [ ]:
# Visualising distribution of data
from pandas.plotting import scatter_matrix

THE SCATTER matrix is plotting each of the columns specified against each other column.

Each cell of the matrix :

shows the relationship between two variables

Displays a histogram on the diagonal for the distribution of each variable

In [ ]:
sns.pairplot(selectdata[['HTGD','ATGD','HTP','ATP','DiffFormPts','DiffGD', 'FTR']], hue='FTR')
plt.show()


In [ ]:
X_all = selectdata.copy().drop('FTR',axis=1)
y_all = selectdata['FTR']
#Standarising the data
from sklearn.preprocessing import scale
#Centre to the mean
cols = [['HTGD','ATGD','HTP','ATP','DiffFormPts','DiffGD']]
for col in cols:
    X_all[col] = scale(X_all[col])

print(X_all)


In [ ]:
# Fit and transform the categorical columns
encoded_array = encoder.fit_transform(X_all[["HM1","HM2","HM3","AM1","AM2","AM3"]])
# Get correct feature names using get_feature_names_out()
encoded_columns = encoder.get_feature_names_out(["HM1","HM2","HM3","AM1","AM2","AM3"])
#Convert the encoded data into a DataFrame with the correct column names
nominal = pd.DataFrame(encoded_array.toarray(),
                       columns=encoded_columns) 
print(encoded_array.toarray())
print(nominal)

In [ ]:
X_all_num = X_all.copy().drop(columns=["HM1","HM2","HM3","AM1","AM2","AM3"],axis=1)
X_all_final = pd.concat([X_all_num.reset_index(drop=True), nominal.reset_index(drop=True)],axis=1)
X_all_final

Spliting the dataset

In [ ]:
from sklearn.model_selection import train_test_split

# Shuffle and split the dataset into training and testing set.
X_train, X_test, y_train, y_test = train_test_split(X_all_final, y_encoded, 
                                                    test_size = 0.3,
                                                    random_state = 2,
                                                    stratify = y_encoded)

Applying Logitstic regression

In [ ]:
# Fitting Logistic Regression to the Training set
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)  # au lieu de 100
model.fit(X_train, y_train)

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, Y_pred)
print(f"Accuracy : {accuracy:.2f}")


In [ ]:
from sklearn.metrics import confusion_matrix


cm = confusion_matrix(y_test, Y_pred, labels=[0, 1, 2])  # H=0, D=1, A=2
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['H', 'D', 'A'], yticklabels=['H', 'D', 'A'])
plt.xlabel("Prédiction")
plt.ylabel("Vérité")
plt.title("Matrice de confusion")
plt.show()


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, Y_pred, target_names=['H', 'D', 'A']))


Applying Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=700,min_samples_split=10,min_samples_leaf=1,random_state=1,n_jobs=-1,oob_score=True,class_weight='balanced')

classifier.fit(X_train,y_train)

In [ ]:
Y_pred = classifier.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, Y_pred, labels=[0, 1, 2]) 

In [ ]:
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['H', 'D', 'A'], yticklabels=['H', 'D', 'A'])
plt.xlabel("Prédiction")
plt.ylabel("Vérité")
plt.title("Matrice de confusion")
plt.show()


In [ ]:
print(classification_report(y_test, Y_pred, target_names=['H', 'D', 'A']))

Applying XGBoost

In [ ]:


from xgboost import XGBClassifier
classifier = XGBClassifier(seed=82)
classifier.fit(X_train, y_train)


In [ ]:
Y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, Y_pred, labels=[0, 1, 2])

In [ ]:
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['H', 'D', 'A'], yticklabels=['H', 'D', 'A'])
plt.xlabel("Prédiction")
plt.ylabel("Vérité")
plt.title("Matrice de confusion")
plt.show()


In [ ]:
print(classification_report(y_test, Y_pred, target_names=['H', 'D', 'A']))

Neural network

In [10]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.utils.class_weight import compute_class_weight # For handling class imbalance


In [6]:
print(tf.__version__)

2.18.1


# Model Performance Comparison

The table below summarizes the performance of the three models (Logistic Regression, Random Forest, XGBoost) on the test set. Metrics include accuracy, precision, recall, and F1-score for each class (H, D, A).

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Logistic Regression
logit_pred = model.predict(X_test)
logit_acc = accuracy_score(y_test, logit_pred)
logit_report = classification_report(y_test, logit_pred, target_names=['H', 'D', 'A'], output_dict=True)

# Random Forest
rf_pred = classifier.predict(X_test)
rf_acc = accuracy_score(y_test, rf_pred)
rf_report = classification_report(y_test, rf_pred, target_names=['H', 'D', 'A'], output_dict=True)

# XGBoost
from xgboost import XGBClassifier
xgb = XGBClassifier(seed=82)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
xgb_acc = accuracy_score(y_test, xgb_pred)
xgb_report = classification_report(y_test, xgb_pred, target_names=['H', 'D', 'A'], output_dict=True)


summary = pd.DataFrame({
    'Logistic Regression': [logit_acc, logit_report['H']['f1-score'], logit_report['D']['f1-score'], logit_report['A']['f1-score']],
    'Random Forest': [rf_acc, rf_report['H']['f1-score'], rf_report['D']['f1-score'], rf_report['A']['f1-score']],
    'XGBoost': [xgb_acc, xgb_report['H']['f1-score'], xgb_report['D']['f1-score'], xgb_report['A']['f1-score']]
}, index=['Accuracy', 'F1-score (H)', 'F1-score (D)', 'F1-score (A)'])

print('Model Performance Summary Table:')
display(summary.round(3))

**Interpretation:**
- The model with the highest accuracy and balanced F1-scores across all classes is considered the best performer.
- Review the table above to identify the most performant model for your dataset.

In [3]:
# Verify TensorFlow installation
import tensorflow as tf
print(tf.__version__)

2.18.1
